In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
from tqdm import tqdm, tnrange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import os
from gptzero.model import GPT2PPL
from cleantext import clean
from langdetect import detect
import re

In [2]:
availability = torch.cuda.is_available()
print(availability)
d_count = torch.cuda.device_count()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

True


In [3]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
def contains_alpha(input):
    search_results = re.search('[a-zA-Z]', input)
    if search_results == None:
        return False
    else:
        return True
    return search_results
def isurl(input):   
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, input) is not None 

def printline():
    print("------------------------------------------")

In [4]:
def infer(sentence):
    sentence1 = sentence
    sentence = remove_emojis(sentence)
    sentence = clean(sentence)
    model = GPT2PPL()
    output = model(sentence)
    return output[0]["label"]

In [5]:
limit = 3000
df = pd.read_csv("GPT-wiki-intro.csv")
df = df.drop(["url", "id", "title", "title_len", "wiki_intro_len", "prompt"],axis=1)
df = df.drop(["prompt_tokens", "generated_intro_len"],axis=1)
df = df.drop(["generated_text_tokens"],axis=1)

In [6]:
df = df.iloc[:limit]
df.describe()

,wiki_intro,generated_intro,generated_text
count,3000,3000,3000
unique,3000,3000,3000
top,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,"located in the town of Sexhow, on the Cumbria..."
freq,1,1,1


In [7]:
real = []
gen = []
print("Running Inference")
for index, values in tqdm(df.iterrows()): 
    val_real = infer(values.wiki_intro)
    val_fake = infer(values.generated_intro)
    real.append(val_real)
    gen.append(val_fake)

Running Inference


0it [00:00, ?it/s]

Perplexity 789
Perplexity per line 789.0
Burstiness 789


1it [00:09,  9.60s/it]

Perplexity 1486
Perplexity per line 1486.0
Burstiness 1486
Perplexity 1262
Perplexity per line 1262.0
Burstiness 1262


...

Perplexity 472
Perplexity per line 472.0
Burstiness 472
Perplexity 1005
Perplexity per line 1005.0
Burstiness 1005


2999it [7:13:32,  9.92s/it]

Perplexity 2142
Perplexity per line 2142.0
Burstiness 2142
Perplexity 563
Perplexity per line 563.0
Burstiness 563


3000it [7:13:42,  8.67s/it]

Perplexity 56
Perplexity per line 56.0
Burstiness 56


In [8]:
def real_gen_check(real, gen, total):
    realperc = 0
    for i in real:
        realperc+=i
    genperc = 0
    for i in gen:
        genperc+=i
    print("Real is ", realperc/total, "Generated is", 1-(genperc/total) )

real_gen_check(real, gen, limit)

Real is  0.9913333333333333 Generated is 0.021333333333333315
